# Create the code to test REPUBLIC with lots of PLATO-like light curves
##### Oscar Barragán, Nov 2023

In [ ]:
#Load the libraries
from __future__ import division, absolute_import, print_function
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline
from matplotlib import gridspec
#Be sure you have citlalicue installed, if not, install it with
#pip install citlalicue
from citlalicue.citlalicue import citlali
#The republic module is inside this directory
import republic
import seaborn as sns
sns.set_theme(style="white")
sns.set_context("paper")

In [ ]:
#DEFINE THE MULTICAMERA CONFIGURATION HERE
#I create the data set for 24 cameras, but later it can be changed to 
#run with less cameras taking subsets of the 24 cameras array
J = 24  # number of cameras
N = 4    # number of trends per camera
K = 1000 # number of observations
N_lcs = 1000 #Number of light curves
kepler_quarter = 10 #Select the Kepler quarter that we are using to obtain the Cotrending Basis Vectors
t = np.linspace(0,90,K)  #Time span to simulate the observations
np.random.seed(1) #Random seed to ensure reproducibility between different runs of this code

## Let us create the light curves

In [ ]:
#Ranges to create the light curves

#QP kernel hyper parameters
Amps = [1e-5,5e-3]
les  = [10,1000]
lps  = [0.1,2]
Pgps = [3,30]

#Planet parameters
t0s  = [0,5]
Ps   = [0,10]
bs   = [0,1]
ars  = [1.5,50]
rps  = [0.01,0.1]

In [ ]:
#Create the light curves
#lcs is a list that contain all the light curve objects
lcs = [None]*N_lcs
for i in tqdm(range(N_lcs)):
    #Let us create a light curve using citlalicue
    lc = citlali(time=t) 
    #In this case, we just need to call the add_spots method
    Amp = np.random.uniform(*Amps)
    le  = np.random.uniform(*les)
    lp  = np.random.uniform(*lps)
    Pgp = np.random.uniform(*Pgps)
    lc.add_spots(QP=[Amp,le,lp,Pgp])
    T0 = np.random.uniform(*t0s)
    P  = np.random.uniform(*Ps)
    b  = np.random.uniform(*bs)
    a  = np.random.uniform(*ars)
    rp = np.random.uniform(*rps)
    u1 = 0
    u2 = 0
    #Let us create a list with all the planet parameters
    planet_parameters = [T0,P,b,a,rp,u1,u2]
    #Let us add the planet by calling the add_transits method
    lc.add_transits(planet_parameters=planet_parameters,planet_name='b')
    #lc.plot(fsx=20,fsy=5)

    #Let's store the current light curve instance in the master list
    lcs[i] = lc

In [ ]:
#Check that the instances have light curves
lcs[9].plot()

## Time to create the trends that will mimic the camera-like systematics
We are creating a master set with 24 cameras, and uncorrelated systatics, then using this we can easily create trends with correlated systematics and/or less cameras

In [ ]:
#Let us create the different sets of systematic treds
#create_kepler_CBVs is a function inside republic.py
T_master = republic.create_kepler_CBVs(t,quarter=kepler_quarter,N_cameras=J,ndata=K,N_trends=N,plot_cbvs=True)

In [ ]:
#Create a plot with the systematics of different cameras
fig = plt.figure(1,figsize=(15,10))
gs = gridspec.GridSpec(nrows=3,ncols=2)
gs.update(hspace=0.025)
gs.update(wspace=0.01)
cameras = [0,1,2,3,4,5]
#
for i,cam in enumerate(cameras):
    plt.subplot(gs[i])
    for j in range(N):
        if i == 0:
            plt.plot(t,T_master[cam,:,j],alpha=0.9,label='Trend '+str(j+1),lw=1)
        else:
            plt.plot(t,T_master[cam,:,j],alpha=0.9,lw=1)
    if i == 0:
        plt.legend(loc=1,ncols=4,frameon=True)
    if i%gs.ncols != 0:
        plt.tick_params(axis='y',labelleft=False,left=False)
    else:
        plt.ylabel('Value')
    plt.xlabel('Time [days]')
    plt.xlim(0.01,89.9)
    plt.ylim(-0.59,0.59)

out_file = 'trends.pdf'
plt.savefig(out_file,bbox_inches='tight',dpi=300)

## Time to platorize the light curves

In [ ]:
#LEt us create the PLATO-like data for all cameras adding a new attribute to the lcs[i] instances
#All stars will have the same trends
sig = 0.0005
for i in range(N_lcs):
    lcs[i].plato = republic.platorize(lcs[i].flux,T_master,J,N,K,sig=sig)

In [ ]:
#Plot only one light curve to check that everything is OK
plt.figure(figsize=(15,5))
n = 0 #Let us plot the first light curve
for i in range(J):
    plt.plot(t,lcs[n].plato[i]-i*0.01,'.',alpha=0.3)
plt.xlim(t.min(),t.max())
plt.show()

In [ ]:
#Tests can be done with less light curves to make this notebook run faster
N_l = 100
#To run with the whole set of light curves uncomment the next line
#N_l = N_lcs

## Let us correct the light curves using republic with 6, 12 and 24 cameras

In [ ]:
#Run republic for all cameras
for i in tqdm(range(N_l)):
    #6 cameras
    #Let us do the republic magic correcting with a different number of cameras each time
    ncam=6
    T_use = T_master[:ncam,:,:] 
    sigma = np.zeros((ncam,K)) + sig
    lcs[i].a_6, lcs[i].w_6, lcs[i].B_6 = republic.republic_solve(lcs[i].plato[0:ncam], T_use, sigma)
    #12 cameras
    ncam=12
    T_use = T_master[:ncam,:,:] 
    sigma = np.zeros((ncam,K)) + sig
    lcs[i].a_12, lcs[i].w_12, lcs[i].B_12 = republic.republic_solve(lcs[i].plato[0:ncam], T_use, sigma)
    #24 cameras
    ncam=24
    T_use = T_master[:ncam,:,:] 
    sigma = np.zeros((ncam,K)) + sig
    lcs[i].a_24, lcs[i].w_24, lcs[i].B_24 = republic.republic_solve(lcs[i].plato[0:ncam], T_use, sigma)

In [ ]:
nl = 5
fig = plt.figure(1,figsize=(15,5),rasterized=True)
plt.plot(lcs[nl].time,(lcs[nl].a_6 -lcs[nl].flux)*1e3,'.',label= '6 Cameras',lw=0.5)
plt.plot(lcs[nl].time,(lcs[nl].a_12-lcs[nl].flux)*1e3,'.',label='12 Cameras',lw=0.5)
plt.plot(lcs[nl].time,(lcs[nl].a_24-lcs[nl].flux)*1e3,'.',label='24 Cameras',lw=0.5)
plt.ylabel('Residuals [ppt]')
plt.xlabel('Time [days]')
plt.legend(frameon=True)
plt.xlim(t.min(),t.max())
out_file = 'cameras_differences.pdf'
plt.savefig(out_file,bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(lcs[nl].time,lcs[nl].a_6 ,'.',label='6')
plt.plot(lcs[nl].time,lcs[nl].a_12+0.05,'.',label='12')
plt.plot(lcs[nl].time,lcs[nl].a_24+0.09,'.',label='24')
plt.xlim(t.min(),t.max())
plt.legend(frameon=True)

## LEt us test for light curves with different levels of white noise

In [ ]:
#LEt us create the PLATO-like data for all cameras adding a new attribute to the lcs[i] instances
#All stars will have the same trends
sig = 0.005
for i in range(N_l):
    lcs[i].plato_wn5ppt = republic.platorize(lcs[i].flux,T_master,J,N,K,sig=sig)
#50 times more white noise
sig = 0.025
for i in range(N_l):
    lcs[i].plato_wn25ppt = republic.platorize(lcs[i].flux,T_master,J,N,K,sig=sig)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(lcs[0].plato[0],'.')
plt.plot(lcs[0].plato_wn5ppt[0],'.')
plt.plot(lcs[0].plato_wn25ppt[0],'.')

In [ ]:
#let us correct the light curves with REPUBLIC
#Run republic for all cameras
for i in tqdm(range(N_l)):
    #6 cameras
    #Let us do the republic magic correcting with a different number of cameras each time
    ncam=6
    T_use = T_master[:ncam,:,:] 
    sig = 0.005
    sigma = np.zeros((ncam,K)) + sig
    lcs[i].a_6_wn5ppt, lcs[i].w_6_wn5ppt, lcs[i].B_6_wn5ppt = republic.republic_solve(lcs[i].plato_wn5ppt[0:ncam], T_use, sigma)
    sig = 0.025
    sigma = np.zeros((ncam,K)) + sig
    lcs[i].a_6_wn25ppt, lcs[i].w_6_wn25ppt, lcs[i].B_6_wn25ppt = republic.republic_solve(lcs[i].plato_wn25ppt[0:ncam], T_use, sigma)
    #12 cameras
    ncam=12
    T_use = T_master[:ncam,:,:] 
    sig = 0.005
    sigma = np.zeros((ncam,K)) + sig
    lcs[i].a_12_wn5ppt, lcs[i].w_12_wn5ppt, lcs[i].B_12_wn5ppt = republic.republic_solve(lcs[i].plato_wn5ppt[0:ncam], T_use, sigma)
    sig = 0.025
    sigma = np.zeros((ncam,K)) + sig
    lcs[i].a_12_wn25ppt, lcs[i].w_12_wn25ppt, lcs[i].B_12_wn25ppt = republic.republic_solve(lcs[i].plato_wn25ppt[0:ncam], T_use, sigma)
    #24 cameras
    ncam=24
    T_use = T_master[:ncam,:,:] 
    sig = 0.005
    sigma = np.zeros((ncam,K)) + sig
    lcs[i].a_24_wn5ppt, lcs[i].w_24_wn5ppt, lcs[i].B_24_wn5ppt = republic.republic_solve(lcs[i].plato_wn5ppt[0:ncam], T_use, sigma)
    sig = 0.025
    sigma = np.zeros((ncam,K)) + sig
    lcs[i].a_24_wn25ppt, lcs[i].w_24_wn25ppt, lcs[i].B_24_wn25ppt = republic.republic_solve(lcs[i].plato_wn25ppt[0:ncam], T_use, sigma)

In [ ]:
#Let us compare the residuals for all the cases
nlc = 0
plt.figure(figsize=(15,5),rasterized=True)
plt.plot(lcs[nlc].time,(lcs[nlc].a_24        -lcs[nlc].flux)*1e3,'.',color='C0',alpha=0.8,
         zorder=2,label='original white noise = 0.5 ppt')
plt.plot(lcs[nlc].time,(lcs[nlc].a_24_wn5ppt -lcs[nlc].flux)*1e3,'.',color='C1',alpha=0.8,
         zorder=1,label='original white noise = 5 ppt')
plt.plot(lcs[nlc].time,(lcs[nlc].a_24_wn25ppt-lcs[nlc].flux)*1e3,'.',color='C2',alpha=0.8,
         zorder=0,label='original white noise = 25 ppt')
#
plt.xlim(lcs[nlc].time.min(),lcs[nlc].time.max())
plt.legend(frameon=True,loc=1)

plt.ylabel('Residuals [ppt]')
plt.xlabel('Time [days]');
out_file = 'whitenoise_differences.pdf'
plt.savefig(out_file,bbox_inches='tight',dpi=300)

## Let's correct the light curves with a PDC-LS-like algorithm

In [ ]:
#light curve by light curve
for n in tqdm(range(N_l)):
    lcs[n].pdcls = [None]*J
    #camera by camera
    for j in range(J):
        #Let us shift the fluxes to zero to be able to do PDCLS 
        basis = np.array(T_master[j,:,:])
        lcs[n].pdcls[j] = republic.PDCLS(lcs[n].plato[j]-np.mean(lcs[n].plato[j]),basis.T)
    #Combine all the light curves 
    lcs[n].pdcls_mean = np.mean(lcs[n].pdcls,axis=0)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(lcs[nlc].pdcls_mean-lcs[nlc].flux+np.mean(lcs[nlc].flux))
plt.plot(lcs[nlc].a_24-lcs[nlc].flux)

### Create the plot for the paper

In [ ]:
fig = plt.figure(1,figsize=(14,17))
gs = gridspec.GridSpec(nrows=4,ncols=1)
gs.update(hspace=0.025)
#
plt.subplot(gs[0],rasterized=True)
plt.plot(lcs[nlc].time,lcs[nlc].flux,'k-',label='True signal')
plt.xlim(lcs[nlc].time.min(),lcs[nlc].time.max())
plt.tick_params(axis='x', which='both', direction='in',labelbottom=False)
plt.ylabel('Flux')
plt.annotate('a)',(0.1,0.91),xycoords='subfigure fraction')
#
plt.subplot(gs[1],rasterized=True)
for j in range(J):
    if j == 0:
        plt.plot(lcs[nlc].time,lcs[nlc].plato[j],'o',lw=0.75,alpha=.5, label = 'LC cameras',markersize=2)
    else:
        plt.plot(lcs[nlc].time,lcs[nlc].plato[j]-j*0.01,'o',lw=0.75,alpha=.5,markersize=2)
plt.xlim(lcs[nlc].time.min(),lcs[nlc].time.max())
plt.tick_params(axis='x', which='both', direction='in',labelbottom=False)
plt.ylabel('Flux + offset')
plt.annotate('b)',(0.1,0.7),xycoords='subfigure fraction')
#
plt.subplot(gs[2],rasterized=True)
plt.plot(lcs[nlc].time,lcs[nlc].pdcls_mean+np.mean(lcs[nlc].flux),'C0.',label='PDCLS',alpha=0.75)
plt.plot(lcs[nlc].time,lcs[nlc].a_24,'C1.',label='REPUBLIC',alpha=0.75)
plt.plot(lcs[nlc].time,lcs[nlc].flux,'k-',label='True signal')
plt.xlim(lcs[nlc].time.min(),lcs[nlc].time.max())
plt.tick_params(axis='x', which='both', direction='in',labelbottom=False)
plt.ylabel('Flux')
plt.annotate('c)',(0.1,0.47),xycoords='subfigure fraction')
#
plt.subplot(gs[3],rasterized=True)
plt.plot(lcs[nlc].time,lcs[nlc].pdcls_mean-lcs[nlc].flux+np.mean(lcs[nlc].flux),'C0.',
         label='PDCLS $-$ True',alpha=0.75)
plt.plot(lcs[nlc].time,lcs[nlc].a_24-lcs[nlc].flux,'C1.',label='REPUBLIC $-$ True',alpha=0.75)
plt.xlim(lcs[nlc].time.min(),lcs[nlc].time.max())
plt.tick_params(axis='x', which='both', direction='in',labelbottom=True)
plt.xlabel('Time [days]')
plt.ylabel('Residuals')
plt.annotate('d)',(0.1,0.23),xycoords='subfigure fraction')
out_file = 'lc.pdf'
plt.savefig(out_file,bbox_inches='tight',dpi=300)
#plt.savefig('republic-broken.png',bbox_inches='tight',dpi=300)

### Extract the trends with PCA

In [ ]:
#Let us do a PCA for one single camera j
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

N_components = 4

#Let us create an array to store the trends obtained with CBVs
T_CBV = np.ones((J,K,N_components)) 

#Here we need all the light curve N_lcs (not N_l) to extract the best CBVs
data = [None]*N_lcs

#camera by camera
for j in tqdm(range(J)):
    #light curve by light curve
    for i in range(N_lcs):
        data[i] = lcs[i].plato[j]
    #Now data has the information for all the light curves for a given camera
    scaler = StandardScaler()
    data_standardized = scaler.fit_transform(np.array(data).T)
    #Apply PCA
    pca = PCA()
    pca_result = pca.fit_transform(data_standardized)
    pca_components = PCA(n_components=N_components)
    #extract the components
    transformed_data = pca_components.fit_transform(data_standardized)
    transformed_data_df = pd.DataFrame(transformed_data)
    #Save the recovered PCA components in the T_CBV array to be used by republic
    for k in range(N_components):
        T_CBV[j,:,k] = transformed_data_df[k] 
        T_CBV[j,:,k] = (T_CBV[j,:,k] - T_CBV[j,:,k].min()) / (T_CBV[j,:,k].max() - T_CBV[j,:,k].min()) - 0.5

In [ ]:
#Let us plot the extracted trends
plt.figure(figsize=(15,5))
for k in range(N_components):
    plt.plot(T_CBV[0,:,k])

In [ ]:
#Create a plot with the systematics of different cameras
fig = plt.figure(1,figsize=(15,10))
gs = gridspec.GridSpec(nrows=3,ncols=2)
gs.update(hspace=0.025)
gs.update(wspace=0.01)
cameras = [0,1,2,3,4,5]
#
for i,cam in enumerate(cameras):
    plt.subplot(gs[i])
    for j in range(N):
        if i == 0:
            plt.plot(t,T_CBV[cam,:,j],alpha=0.9,label='Trend '+str(j+1),lw=1)
        else:
            plt.plot(t,T_CBV[cam,:,j],alpha=0.9,lw=1)
    if i == 0:
        plt.legend(loc=1,ncols=4,frameon=True)
    if i%gs.ncols != 0:
        plt.tick_params(axis='y',labelleft=False,left=False)
    else:
        plt.ylabel('Value')
    plt.xlabel('Time [days]')
    plt.xlim(0.01,89.9)
    plt.ylim(-0.59,0.6)

out_file = 'trends_PCA.pdf'
plt.savefig(out_file,bbox_inches='tight',dpi=300)

In [ ]:
#Combined plot
#Create a plot with the systematics of different cameras
fig = plt.figure(1,figsize=(12,15))
gs = gridspec.GridSpec(nrows=6,ncols=2)
gs.update(hspace=0.025)
gs.update(wspace=0.01)
cameras = [0,1,2,3,4,5]
colors=['C0','C1','C2','C3']
#
for i,cam in enumerate(cameras):
    
#
    plt.subplot(gs[i*2])
    for j in range(N):
        if i == 0:
            plt.plot(t,T_master[cam,:,j],alpha=0.9,label='Trend '+str(j+1),lw=1,ls='-',color=colors[j])
        else:
            plt.plot(t,T_master[cam,:,j],alpha=0.9,lw=1,ls='-',color=colors[j])
        if i == 0:
            plt.legend(loc=1,ncols=4,frameon=True)

        plt.ylabel('Value')
    
    plt.xlabel('Time [days]')
    plt.xlim(0.01,89.9)
    plt.ylim(-0.59,0.6)
#

    plt.subplot(gs[i*2+1])
    for j in range(N):
        if i == 0:
            plt.plot(t,T_CBV[cam,:,j],alpha=0.9,label='Trend '+str(j+1),lw=1,color=colors[j],ls='--')
        else:
            plt.plot(t,T_CBV[cam,:,j],alpha=0.9,lw=1,color=colors[j],ls='--')
    if i == 0:
        plt.legend(loc=1,ncols=4,frameon=True)

    if i*2%gs.ncols == 0:
        plt.tick_params(axis='y',labelleft=False,left=False)
    else:
        plt.ylabel('Value')
        
    plt.xlabel('Time [days]')
    plt.xlim(0.01,89.9)
    plt.ylim(-0.59,0.6)

out_file = 'trends_PCA.pdf'
plt.savefig(out_file,bbox_inches='tight',dpi=300)

In [ ]:
#Now let us correct the light curves with this
ncam=24
for n in tqdm(range(N_l)):
    T_use = T_CBV[:ncam,:,:] 
    sigma = np.zeros((ncam,K)) + sig
    lcs[n].a_pca, lcs[n].w_pca, lcs[n].B_pca = republic.republic_solve(lcs[n].plato[:ncam], T_use, sigma)

In [ ]:
nlc = 0
plt.figure(figsize=(15,5))
plt.plot(lcs[nlc].a_pca)
plt.plot(lcs[nlc].a_24)
plt.plot(lcs[nlc].flux)
plt.plot(lcs[nlc].plato[5])

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(lcs[nlc].a_pca-lcs[nlc].flux)
plt.plot(lcs[nlc].a_24-lcs[nlc].flux)

### How the PDCLS behave with the imperfect trends

In [ ]:
#light curve by light curve
for n in tqdm(range(N_l)):
    lcs[n].pdcls_cbv = [None]*J
    #camera by camera
    for j in range(J):
        #Let us shift the fluxes to zero to be able to do PDCLS 
        basis = np.array(T_CBV[j,:,:])
        lcs[n].pdcls_cbv[j] = republic.PDCLS(lcs[n].plato[j]-np.mean(lcs[n].plato[j]),basis.T)
    #Combine all the light curves 
    lcs[n].pdcls_mean_cbv = np.mean(lcs[n].pdcls_cbv,axis=0)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(lcs[nlc].a_pca-lcs[nlc].flux)
plt.plot(lcs[nlc].pdcls_mean_cbv-lcs[nlc].flux+1)

In [ ]:
fig = plt.figure(1,figsize=(14,7))
gs = gridspec.GridSpec(nrows=2,ncols=1)
gs.update(hspace=0.025)
#
plt.subplot(gs[0],rasterized=True)
plt.plot(lcs[nlc].time,lcs[nlc].a_pca,'C1.',label='REPUBLIC with PCA',alpha=0.75)
plt.plot(lcs[nlc].time,lcs[nlc].pdcls_mean_cbv+1,'C0.',label='PDCLS with PCA',alpha=0.75)
plt.plot(lcs[nlc].time,lcs[nlc].flux,'k-',label='True signal')
plt.xlim(lcs[nlc].time.min(),lcs[nlc].time.max())
plt.tick_params(axis='x', which='both', direction='in',labelbottom=False)
plt.ylabel('Flux')
#
plt.subplot(gs[1],rasterized=True)
plt.plot(lcs[nlc].time,lcs[nlc].a_pca-lcs[nlc].flux,'C1.',label='REPUBLIC with PCA',alpha=0.75)
plt.plot(lcs[nlc].time,lcs[nlc].pdcls_mean_cbv-lcs[nlc].flux+1,'C0.',label='PDCLS with PCA',alpha=0.75)
plt.xlim(lcs[nlc].time.min(),lcs[nlc].time.max())
plt.tick_params(axis='x', which='both', direction='in',labelbottom=True)
plt.xlabel('Time [days]')
plt.ylabel('Residuals')
out_file = 'lc_nonidealcase.pdf'
plt.savefig(out_file,bbox_inches='tight',dpi=300)

# Correlated trends

In [ ]:
#Create correlated trends
#We will make that the first trend in all cameras is identical
T_correlated = np.array(T_master)
for i in range(1,J):
    T_correlated[i,:,0] = T_master[0,:,0]


    
#Check visually that this works
#
#Create a plot with the systematics of different cameras
fig = plt.figure(1,figsize=(15,10))
gs = gridspec.GridSpec(nrows=3,ncols=2)
gs.update(hspace=0.025)
gs.update(wspace=0.01)
cameras = [0,1,2,3,4,5]
#

for i,cam in enumerate(cameras):
    plt.subplot(gs[i])
    for j in range(N):
        if i == 0:
            plt.plot(t,T_correlated[cam,:,j],alpha=0.9,label='Trend '+str(j+1),lw=1)
        else:
            plt.plot(t,T_correlated[cam,:,j],alpha=0.9,lw=1)
    if i == 0:
        plt.legend(loc=1,ncols=4,frameon=True)
    if i%gs.ncols != 0:
        plt.tick_params(axis='y',labelleft=False,left=False)
    else:
        plt.ylabel('Value')
    plt.xlabel('Time [days]')
    plt.xlim(0.01,89.9)

In [ ]:
#Now let us create PLATO light curves using these correlated systematics
#We add a new attribute called plato_corr, similar to plato but with correlated systematics
sig = 0.0005
for i in tqdm(range(N_lcs)):
    lcs[i].plato_corr = republic.platorize(lcs[i].flux,T_correlated,J,N,K,sig=sig)

In [ ]:
#Now let us apply republic to this
#Run republic for 24 cameras
for i in tqdm(range(N_l)):
    #24 cameras
    ncam=24
    T_use = T_correlated[:ncam,:,:] 
    sigma = np.zeros((ncam,K)) + sig
    lcs[i].a_corr, lcs[i].w_corr, lcs[i].B_corr = republic.republic_solve(lcs[i].plato_corr[0:ncam], T_use, sigma)

In [ ]:
plt.plot(lcs[nlc].time,lcs[nlc].flux,'k-',label='True signal')
plt.plot(lcs[nlc].time,lcs[nlc].a_corr,'C3.',label='REPUBLIC $-$ True',alpha=0.5)

## Now we will use the PDC-like algorithm to correct with the correlated systematics

In [ ]:
#light curve by light curve
for n in tqdm(range(N_l)):
    lcs[n].pdcls_corr = [None]*J
    #camera by camera
    for j in range(J):
        #Let us shift the fluxes to zero to be able to do PDCLS 
        basis = np.array(T_correlated[j,:,:])
        lcs[n].pdcls_corr[j] = republic.PDCLS(lcs[n].plato[j]-np.mean(lcs[n].plato_corr[j]),basis.T)
    #Combine all the light curves 
    lcs[n].pdcls_mean_corr = np.mean(lcs[n].pdcls_corr,axis=0)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(lcs[nlc].pdcls_mean_corr-lcs[nlc].flux+np.mean(lcs[nlc].flux))
plt.plot(lcs[nlc].a_corr-lcs[nlc].flux)

## Let us extract the correlated CBVs from the correlated case

In [ ]:
N_components = 4

#Let us create an array to store the trends obtained with CBVs
T_CBV_corr = np.ones((J,K,N_components)) 

#Here we need all the light curve N_lcs (not N_l) to extract the best CBVs
data = [None]*N_lcs

#camera by camera
for j in tqdm(range(J)):
    #light curve by light curve
    for i in range(N_lcs):
        data[i] = lcs[i].plato_corr[j]
    #Now data has the information for all the light curves for a given camera
    scaler = StandardScaler()
    data_standardized = scaler.fit_transform(np.array(data).T)
    #Apply PCA
    pca = PCA()
    pca_result = pca.fit_transform(data_standardized)
    pca_components = PCA(n_components=N_components)
    #extract the components
    transformed_data = pca_components.fit_transform(data_standardized)
    transformed_data_df = pd.DataFrame(transformed_data)
    #Save the recovered PCA components in the T_CBV array to be used by republic
    for k in range(N_components):
        T_CBV_corr[j,:,k] = transformed_data_df[k] 
        T_CBV_corr[j,:,k] = (T_CBV_corr[j,:,k] - T_CBV_corr[j,:,k].min()) / (T_CBV_corr[j,:,k].max() - T_CBV_corr[j,:,k].min()) - 0.5

In [ ]:
#Create a plot with the systematics of different cameras
fig = plt.figure(1,figsize=(15,10))
gs = gridspec.GridSpec(nrows=3,ncols=2)
gs.update(hspace=0.025)
gs.update(wspace=0.01)
cameras = [0,1,2,3,4,5]
#

for i,cam in enumerate(cameras):
    plt.subplot(gs[i])
    for j in range(N):
        if i == 0:
            plt.plot(t,T_CBV_corr[cam,:,j],alpha=0.9,label='Trend '+str(j+1),lw=1)
        else:
            plt.plot(t,T_CBV_corr[cam,:,j],alpha=0.9,lw=1)
    if i%gs.ncols != 0:
        plt.tick_params(axis='y',labelleft=False,left=False)
    else:
        plt.ylabel('Value')
    plt.xlabel('Time [days]')
    plt.xlim(0.01,89.9)

In [ ]:
#Now let us apply republic to this
#Run republic for 24 cameras
for i in tqdm(range(N_l)):
    #24 cameras
    ncam=24
    T_use = T_CBV_corr[:ncam,:,:] 
    sigma = np.zeros((ncam,K)) + sig
    lcs[i].a_corr_cbv, lcs[i].w_corr_cbv, lcs[i].B_corr_cbv = republic.republic_solve(lcs[i].plato_corr[0:ncam], T_use, sigma)

In [ ]:
plt.plot(lcs[nlc].time,lcs[nlc].flux,'k-',label='True signal')
plt.plot(lcs[nlc].time,lcs[nlc].a_corr_cbv,'C3.',label='REPUBLIC $-$ True',alpha=0.5)

In [ ]:
plt.plot(lcs[nlc].time,lcs[nlc].a_corr_cbv-lcs[nlc].flux,'C3.',label='REPUBLIC $-$ True',alpha=0.5)

### LET's make the plot of the correlated trends

In [ ]:
#Combined plot
#Create a plot with the systematics of different cameras
fig = plt.figure(1,figsize=(12,15))
gs = gridspec.GridSpec(nrows=6,ncols=2)
gs.update(hspace=0.025)
gs.update(wspace=0.01)
cameras = [0,1,2,3,4,5]
colors=['C0','C1','C2','C3']
#
for i,cam in enumerate(cameras):
    
#
    plt.subplot(gs[i*2])
    for j in range(N):
        if i == 0:
            plt.plot(t,T_correlated[cam,:,j],alpha=0.9,label='Trend '+str(j+1),lw=1,ls='-',color=colors[j])
        else:
            plt.plot(t,T_correlated[cam,:,j],alpha=0.9,lw=1,ls='-',color=colors[j])
        if i == 0:
            plt.legend(loc=1,ncols=4,frameon=True)

        plt.ylabel('Value')
    
    plt.xlabel('Time [days]')
    plt.xlim(0.01,89.9)
    plt.ylim(-0.59,0.6)
#

    plt.subplot(gs[i*2+1])
    for j in range(N):
        if i == 0:
            plt.plot(t,T_CBV_corr[cam,:,j],alpha=0.9,label='Trend '+str(j+1),lw=1,color=colors[j],ls='--')
        else:
            plt.plot(t,T_CBV_corr[cam,:,j],alpha=0.9,lw=1,color=colors[j],ls='--')
    if i == 0:
        plt.legend(loc=1,ncols=4,frameon=True)

    if i*2%gs.ncols == 0:
        plt.tick_params(axis='y',labelleft=False,left=False)
    else:
        plt.ylabel('Value')
        
    plt.xlabel('Time [days]')
    plt.xlim(0.01,89.9)
    plt.ylim(-0.59,0.6)

out_file = 'trends_correlated.pdf'
plt.savefig(out_file,bbox_inches='tight',dpi=300)

In [ ]:
fig = plt.figure(1,figsize=(14,7))
gs = gridspec.GridSpec(nrows=2,ncols=1)
gs.update(hspace=0.025)
#
plt.subplot(gs[0],rasterized=True)
plt.plot(lcs[nlc].time,lcs[nlc].a_corr_cbv,'C0.',label='REPUBLIC with PCA',alpha=0.75)
plt.plot(lcs[nlc].time,lcs[nlc].pdcls_mean_corr+1,'C1.',label='PDCLS with PCA',alpha=0.75)


#NEED TO ADD CODE TOCOMPUTE THE LS CORRECTION
plt.plot(lcs[nlc].time,lcs[nlc].flux,'k-',label='True signal')
plt.xlim(lcs[nlc].time.min(),lcs[nlc].time.max())
plt.tick_params(axis='x', which='both', direction='in',labelbottom=False)
plt.ylabel('Flux')
#
plt.subplot(gs[1],rasterized=True)
plt.plot(lcs[nlc].time,lcs[nlc].a_corr_cbv-lcs[nlc].flux,'C0.',label='REPUBLIC with PCA',alpha=0.75)
plt.plot(lcs[nlc].time,lcs[nlc].pdcls_mean_corr-lcs[nlc].flux+1,'C1.',label='PDCLS with PCA',alpha=0.75)
plt.xlim(lcs[nlc].time.min(),lcs[nlc].time.max())
plt.tick_params(axis='x', which='both', direction='in',labelbottom=True)
plt.xlabel('Time [days]')
plt.ylabel('Residuals')
out_file = 'lc_correlated.pdf'
plt.savefig(out_file,bbox_inches='tight',dpi=300)